Release VERSION 230109

<h1>RSoXS plan simulator</h1>
this is a full stack of the SST scan system, from the loader through to the low level acquisition code, all simulated here
run all the hidden cells first!

In [1]:
!pip install -e /nsls2/users/alevin/repos/rsoxs_scans

Obtaining file:///nsls2/users/alevin/repos/rsoxs_scans
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached zict-2.2.0-py2.py3-none-any.whl (23 kB)
  Building editable for rsoxs-scans (pyproject.toml) ... done
  Created wheel for rsoxs-scans: filename=rsoxs_scans-0.post184.dev0+g4a1a0b6-0.editable-py3-none-any.whl size=4664 sha256=012edef8b29e1d67a42bc697bf4ae8015853220900d85c29fb56d6499dc2b8fb
  Stored in directory: /tmp/pip-ephem-wheel-cache-o0fwib43/wheels/05/fa/82/93883113d790c18a44691574d2bc31622b743713e11acfdd31
Successfully built rsoxs-scans
  Attempting uninstall: zict
    Found existing installation: zict 3.0.0
    Uninstalling zict-3.0.0:
      Successfully uninstalled zict-3.0.0


## Restart kernel!

In [1]:
%load_ext autoreload
# %autoreload 2

from rsoxs_scans.spreadsheets import load_samplesxlsx, save_samplesxlsx
from rsoxs_scans.acquisition import dryrun_bar, get_acq_details
import pathlib

In [2]:
userPath = pathlib.Path('/nsls2/users/alevin')
sheetsPath = userPath.joinpath('rsoxs_suite/imgs_analysis/sample_bars')
sheetsPath.exists()

True

In [18]:
# test your .xlsx file here
# load the example bar
version = 'v8'
bar = load_samplesxlsx(sheetsPath.joinpath(f'Levin-Sample_Bar_{version}.xlsx'))

spreadsheet version is 2023-1 Version 1.0
Reading sheet Acquisitions


In [15]:
save_samplesxlsx(bar, name=f'saved_bar_{version}', path=f'{str(sheetsPath)}/') # test writing to a temporary file

In [16]:
# bar2 = load_samplesxlsx("test2.xlsx") # test loading from a temporary file

In [19]:
# dry run it
outputs = dryrun_bar(bar,['apriority'],group='all')

___________________________________________________________________
Acquisition # 0 from Group: 
	Sample Name: PM6_Si
	Sample id: TEY1
	Project: TEY_NEXAFS

Summary: load TEY1 with config WAXSNEXAFS, run nexafs with priority(Sample: , Acquisition: 1)
	Starts @ 0:00:00 takes 0:16:20 (+120 seconds for configuration change)

> Step 0: load configuration WAXSNEXAFS
> Step 1: load sample PM6_Si
> Step 2: set Diode range to high
> Step 3: calculating a lab-frame polarization of 0.0 from the sample_frame polarization 
  input of 20.0 and a sample angle 20.0
> Step 4: setting sample angle to 20.0
 setting polarization to 0.0
 setting grating to rsoxs
 fly nexafs scanning from 250.0 eV to 350.0 eV on the rsoxs l/mm grating
    at speeds from 0.2 to 1.0 ev/second
> Step 5: calculating a lab-frame polarization of 22.8381407833122 from the sample_frame polarization 
  input of 30.0 and a sample angle 20.0
> Step 6: setting sample angle to 20.0
 setting polarization to 22.8381407833122
 setting gra

In [66]:
# Get more details on a given acquisition
get_acq_details(40,outputs)

--------------------------------------------------
>Step: 0
--------------------------------------------------
{
    "description": "load configuration WAXS\n",
    "action": "load_configuration",
    "kwargs": {
        "configuration": "WAXS"
    },
    "queue_step": 0,
    "acq_index": 40
}
--------------------------------------------------
>Step: 1
--------------------------------------------------
{
    "description": "load sample Y6_2000_directSiN\n",
    "action": "load_sample",
    "kwargs": {
        "sample": {
            "bar_name": "AndrewBar",
            "sample_id": "TRMSN9",
            "sample_name": "Y6_2000_directSiN",
            "project_name": "TRMSN_NEXAFS_and_RSoXS",
            "institution": "CUBLDER",
            "proposal_id": 311130,
            "bar_spot": "7A",
            "front": true,
            "grazing": false,
            "angle": 0,
            "height": 0.2,
            "sample_desc": "",
            "project_desc": "",
            "sample_prior

In [59]:
# get the set of all actions
actions = set()
for acq in outputs:
    for step in acq['steps']:
        actions.add(step['action'])
print(actions)

{'spiral_scan_core', 'message', 'diode_high', 'load_configuration', 'load_sample', 'nexafs_scan_core', 'rsoxs_scan_core'}


In [60]:
arguments = {}
for action in actions:
    keys = set()
    for acq in outputs:
        for step in acq['steps']:
            if step['action'] == action:
                if 'kwargs' in step:
                    for kwarg in step['kwargs']:
                        keys.add(kwarg)
    arguments[action]=keys
    print("{:<18} : {}".format(action,keys))

spiral_scan_core   : {'uid', 'enscan_type', 'md', 'priority', 'configuration', 'type', 'angle', 'sample_id', 'stepsize', 'exposure', 'dets', 'diameter', 'pol', 'energy', 'group', 'grating', 'Parameter/ Index'}
message            : set()
diode_high         : set()
load_configuration : {'configuration'}
load_sample        : {'sample'}
nexafs_scan_core   : {'uid', 'enscan_type', 'md', 'priority', 'configuration', 'type', 'angle', 'sample_id', 'scan_params', 'pol', 'group', 'cycles', 'grating', 'Parameter/ Index'}
rsoxs_scan_core    : {'uid', 'polarizations', 'temps_with_locations', 'md', 'configuration', 'priority', 'enscan_type', 'type', 'sample_id', 'times', 'temp_wait', 'locations', 'dets', 'energies', 'group', 'grating', 'Parameter/ Index'}


<h1>Examples of edges, ratios, frames

In [ ]:
# totally manual simple scan
from rsoxs_scans.constructor import get_energies
energies = get_energies((1850,1930),100)

In [ ]:
# precise energy list
# you can fool the algorithm if you put 1 as the number of frames it will only take one frame at each of the thresholds, including the last one, so you can sort of treat this as just an energy list
# as a corollary, you can't have a total number of frames less than the thresholds that you give it in edges (it will at a minimum always take the threshold values)
energies = get_energies((250,250,250,340,340,341,280,281.45,500,500),1)

In [ ]:
# use a default
energies = get_energies('Nitrogen')

In [ ]:
# use a default with a different name
energies = get_energies('c')

In [ ]:
# use a different built in interval set
energies = get_energies('c','full','carbon nonaromatic')

In [ ]:
# get a very short version of this scan
energies = get_energies('c','very short','carbon nonaromatic')

In [ ]:
# I don't have time for this, I only want ~10 energies
energies = get_energies('c',10,'carbon nonaromatic')

In [ ]:
# also, I made single entry energies also work
energies = get_energies(280)

In [ ]:
# also option to have frames as a list of exact numbers needed in each region
energies = get_energies('c',[2,5,10,40,5,2])

In [ ]:
# putting in zero effectively removes that region
energies = get_energies('c',[2,5,10,0,5,2])

In [ ]:
from rsoxs_scans.constructor import construct_exposure_times
times,_ = construct_exposure_times(get_energies([270,280,290,400],500,[2,1,10]),1)
len(times)

In [ ]:
from rsoxs_scans.constructor import get_nexafs_scan_params
get_nexafs_scan_params('n','normal')

<h1>Example of complex Exposure times

In [ ]:
construct_exposure_times(get_energies((1850,1930),100),[2,("between",1870,1900.0),4,("greater_than",1920.0),1],80)

In [ ]:
isinstance('asdf',str)